# Introdução às matrizes esparsas

<br>

Vicente Sobrinho

I Encontro Regional de Matemática  
UFCA | Centro de Ciências e Tecnologia  
<font size=4pt>Juazeiro do Norte, 7 de outubro de 2024</font>

## Metas de hoje


Ao concluir este minicurso, espera-se que você saiba o seguinte sobre matrizes esparsas:

* definições, propriedades e origens
* esquemas de armazenamento compactos
* operações matriciais básicas
* manipulação de matrizes esparsas com a SciPy

## O que é uma matriz esparsa?

Embora não haja consenso quanto a uma definição, algumas caracterizações são aceitas.

> "The matrix may be sparse, either with the non-zero elements concentrated on a narrow band centered on the diagonal or alternatively they may be distributed in a less systematic manner. We shall refer to a matrix as dense if the percentage of zero elements or its distribution is such as to make it uneconomic to take advantage of their presence."  
(Wilkinson, 1971)

## O que é uma matriz esparsa?

Uma matriz é dita **esparsa** quando possui poucos coeficientes não nulos, quando comparado à quantidade total de coeficientes.

## O que é uma matriz esparsa?

Uma matriz $m \times n$ é esparsa quando possui $nnz = O\left(\min\!\left\{m, n\right\} \right)$ coeficientes não nulos.

## Tipos de esparsidade

* estruturada
* não-estruturada

In [ ]:
# Uma função auxiliar para plotar os padrões de esparsidade
import matplotlib.pyplot as plt
from scipy.io import mmread

def plota(arquivo):
    matriz = mmread(arquivo)
    plt.spy(matriz.toarray(), markersize = 0.5) 
    plt.show()

## Estruturada

In [ ]:
plota('Matrizes/sherman5.mtx.gz')

## Estruturada

In [ ]:
plota('Matrizes/CurlCurl_0.mtx.gz')

## Não-estruturada

In [ ]:
plota('Matrizes/bp_1000.mtx.gz')

## Não-estruturada

In [ ]:
plota('Matrizes/N_biocarta.mtx.gz')

## O conceito vem de longa data

Já em 1823, Gauss trabalhava com esse tipo de matriz ao resolver sistemas de mínimos quadrados.  
Contudo, [Varga (1962)](https://link.springer.com/book/10.1007/978-3-642-05156-2) foi o primeiro a usar explicitamente o termo *esparso*.

Segundo Saad (2023):
* As primeiras técnicas para lidar com problemas esparsos têm origem na solução de equações diferenciais parciais
* Somente na década de 1960 surgiu a metodologia que conhecemos hoje

## De onde elas vêm?

* **Clássicos:** análise estrutural, dinâmina dos fluidos computacional e outras fontes de equações diferenciais

* **Contemporâneos:**
  - Aprendizado de máquina
  - Deep learning
  - Análise de redes sociais

## A eterna busca por métodos eficientes

Desejamos sempre realizar operações sobre matrizes do modo mais eficiente possível, evitando manipular valores nulos.

Por exemplo,

* Somar duas matrices quadradas de ordem $n$ requer $O(n^2)$ operações

* Somar duas matrizes esparsas $\mathbf{A}$ e $\mathbf{B}$ requer $O(nnz(A) + nnz(B))$

## Estruturas de dados compactas

Há uma dezena de esquemas compactos para o armazenamento de matrizes esparsas. Abordaremos aqui os que considero mais versáteis:

* COO: coordenadas
* CSR: linhas esparsas comprimidas (do inglês, *Compressed Sparse Row*)

A versão transposta do CSR é conhecida como *Compressed Sparse Column* (CSC).

## Denso

* Simples
* Coeficientes armazenados por linha ou coluna
* Todos os zeros são armazenados

## Denso

<img src="Figuras/dns_exemplo.svg" width=1280>

Custo de espaço: $O(n^2)$

## COO

* Simples
* Coeficientes nulos não são armazenados
* Falta de ordenação pode deteriorar algoritmos

## COO

<img src="Figuras/coo_exemplo.svg" width=1280>

Custo de espaço: $O(nnz)$

## CSR

* Ainda pode ser considerado simples
* Não armazena coeficientes nulos
* Operações matriciais rápidas

## CSR

<img src="Figuras/csr_exemplo.svg" width=1280>

Custo de espaço: $O(nnz)$

## Operações matriciais

* vetor (denso/esparso) $\odot$ vetor (denso/esparso)
* matriz (densa/esparsa) $\odot$ vetor (denso/esparso)
* matriz (densa/esparsa) $\odot$ matriz (densa/esparsa)

## Operações matriciais

* <font color="lightgrey">vetor (denso/esparso) $\odot$ vetor (denso/esparso)</font>
* **matriz (**<font color="lightgrey">densa/</font>**esparsa)** $\odot$ **vetor (denso**<font color="lightgrey">/esparso</font>**)**
* <font color="lightgrey">matriz (densa/esparsa) $\odot$ matriz (densa/esparsa)</font>

## Produto matriz-vetor

**Entrada:** $\mathbf{A}_{m\times n}$, $\mathbf{x}_{n\times 1}$  

**Saída:** $\mathbf{y}_{m \times 1} = \mathbf{A} \mathbf{x}$  

## Produto matriz-vetor

*Denso*

In [ ]:
def matvec_denso(A,x):
    m,n = A.shape
    y = np.zeros((m,))
    for i in range(m):
        valor = 0.0
        for j in range(n):
            valor = valor + A[i,j]*x[j]
        y[i] = valor
        
    return y

Custo: $O(n^2)$

## Produto matriz-vetor

*Denso*

In [ ]:
import numpy as np

A = np.array([[1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
              [0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0],
              [0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 4.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 1.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0],
              [0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0]])

In [ ]:
x = np.ones((9,))

## Produto matriz-vetor

*Denso*

In [ ]:
matvec_denso(A,x)

## Produto matriz-vetor

*CSR*

In [ ]:
def matvec_csr(linha,coluna,valor,x):
    m,n = A.shape
    y = np.zeros((m,))
    for i in range(m):
        c = 0.0
        inicio = linha[i]
        fim = linha[i+1]
        for j in range(inicio,fim):
            c = c + valor[j]*x[coluna[j]]
        y[i] = c
        
    return y

$\quad\qquad$ Custo: $O(nnz)$

## Produto matriz-vetor

*CSR*

In [ ]:
from scipy.sparse import csr_matrix

Acsr = csr_matrix(A)
Acsr

## Produto matriz-vetor

*CSR*

In [ ]:
Acsr.indptr

In [ ]:
Acsr.indices

In [ ]:
Acsr.data

## Produto matriz-vetor

*CSR*

In [ ]:
matvec_csr(Acsr.indptr, Acsr.indices, Acsr.data, x)

## Não reinvente a roda!

Se for usar matrizes esparsas em Python, explore as funcionalidades oferecidas pela `scipy`.

Códigos caseiros podem ser muito ineficientes!

## Produto matriz-vetor com a NumPy

*Denso*

In [ ]:
np.dot(A,x)

## Produto matriz-vetor esparso com a SciPy

*CSR*

In [ ]:
Acsr.dot(x)

## Resolução de sistemas lineares esparsos

Há duas grandes classes de métodos generalistas para a solução de sistemas lineares:

* Métodos diretos: baseados na eliminação Gaussiana
* Métodos iterativos: baseados em iterações de ponto fixo

Quando lidamos com sistemas de $> 10^6$ equações, os métodos iterativos geralmente são a primeira escolha.

## Resolução de sistemas lineares esparsos

Na SciPy, diversos desses métodos encontram-se implementados, dos quais merecem destaque:

`spsolve(A, b)`
> Resolve via escalonamento o sistema linear esparso $\mathbf{A}\mathbf{x}=\mathbf{b}$, onde $\mathbf{b}$ pode ser um vetor ou uma matriz

## Resolução de sistemas lineares esparsos

Na SciPy, diversos desses métodos encontram-se implementados, dos quais merecem destaque:

`cg`(A, b)  
> Usa o método iterativo do Gradiente Conjugado para matrizes simétricas, positivas-definidas

## Resolução de sistemas lineares esparsos

Na SciPy, diversos desses métodos encontram-se implementados, dos quais merecem destaque:

`gmres`(A, b)  
> Usa o método iterativo do Resíduo Mínimo Generalizado para matrizes não simétricas

## Resolução de sistemas lineares esparsos

In [ ]:
import scipy.sparse.linalg as sp

soln = sp.spsolve(Acsr,np.ones(9))
soln

In [ ]:
soln = sp.cg(Acsr,np.ones(9))
soln

In [ ]:
soln = sp.gmres(Acsr,np.ones(9))
soln

## Aplicação: equação de Laplace

Considere o seguinte problema de valor de contorno:

$$
u_{xx} + u_{yy} = 0, \qquad 0<x<3, \qquad 0<y<2
$$

$$
u(x,0) = 0, \quad  u(x,2) = 0, \quad u(0,y) = 0, \quad u(3,y)= f(y),
$$

$$
f(y) =
\begin{cases}
y, & 0 \le y \le 1\\
2 - y, & 1 \le y \le 2
\end{cases}
$$

## Solução exata

Aplicando o método da separação de variáveis, obtemos a solução exata:

$$
u(x,y) = \dfrac{8}{\pi^2} \sum_{k=1}^{\infty} \frac{\operatorname{sen}\left(k\pi/2\right)}{k^2\operatorname{senh}\left(3k\pi/2\right)} \operatorname{senh}\left(\frac{k\pi x}{2}\right) \operatorname{sen}\left(\frac{k\pi y}{2}\right)
$$

## Solução exata

In [ ]:
def exata(x,y,N=100):
    s = 0.0
    for k in range(1,N+1):
        num1 = np.sin(k*np.pi/2.0)
        num2 = np.sinh(k*np.pi*x/2.0)
        num3 = np.sin(k*np.pi*y/2.0)
        den  = k*k*np.sinh(3*k*np.pi/2.0)
        s = s + (num1/den)*num2*num3
        
    return (8*s)/(np.pi**2)

## Solução exata

In [ ]:
from matplotlib import cm # para selecionar o mapa de cores a seguir

N = 50
x = np.linspace(0, 3, N+2)
y = np.linspace(0, 2, N+2)
X, Y = np.meshgrid(x, y)
Z = exata(X,Y)

## Solução exata

In [ ]:
# Plotando a solução da equação de Laplace
fig, ax = plt.subplots()
mapa = ax.pcolormesh(X, Y, Z, cmap=cm.coolwarm)
fig.colorbar(mapa, ax=ax)
plt.show()

## Solução exata

In [ ]:
# Plotando a solução da equação de Laplace
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=0, antialiased=False)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

## Método das diferenças finitas

Baseado na série de Taylor:

$$
u(x+h) = u(x) + h u'(x) + \frac{h^2}{2} u''(x)  + \frac{h^3}{6} u'''(x) + \frac{h^4}{24} u^{(4)}(\xi)
$$

## Aproximação da segunda derivada

Somando as expressões:

$$
u(x-h) = u(x) - h u'(x) + \frac{h^2}{2} u''(x)  - \frac{h^3}{6} u'''(x) + \frac{h^4}{24} u^{(4)}(\xi_1)
$$

$$
u(x+h) = u(x) + h u'(x) + \frac{h^2}{2} u''(x)  + \frac{h^3}{6} u'''(x) + \frac{h^4}{24} u^{(4)}(\xi_2)
$$

## Aproximação da segunda derivada

Obtemos:

$$
u''(x) = \frac{u(x+h) - 2 u(x) + u(x-h)}{h^2} - \frac{h^2}{12} u^{(4)}(\xi)
$$

com $\xi_1 \le \xi \le \xi_2$.

## Aproximação da segunda derivada

Portanto:

$$
u''(x) \approx \frac{u(x+h) - 2 u(x) + u(x-h)}{h^2}
$$

## Discretização da equação de Laplace

Discretizando uniformemente o retângulo $[0,3] \times [0,2]$ em $n + 1$ sub-intervalos em cada dimensão, temos

$$
x_i = i\cdot h_1, \qquad i = 0, 1, \dotsc, n+1, \qquad h_1 = \frac{3}{n+1}
$$

$$
y_j = j\cdot h_2, \qquad j = 0, 1, \dotsc, n+1, \qquad h_2 = \frac{2}{n+1}
$$

## Discretização da equação de Laplace

Definindo

$$
u_{i,j} = u(x_i, y_j)
$$

## Discretização da equação de Laplace

podemos escrever:

$$
u_{xx}(x_i,y_j) + u_{yy}(x_i,y_j) \approx \frac{u_{i-1,j} + u_{i+1,j}}{h_1^2}
        - 2 \left( \frac{1}{h_1^2} + \frac{1}{h_2^2}\right) u_{ij}
        + \frac{u_{i,j-1} + u_{i,j+1}}{h_2^2} = 0
$$

## Discretização da equação de Laplace

In [ ]:
# Domínio retangular I x J
I = (0.0, 3.0)
J = (0.0, 2.0)

In [ ]:
# Quantidade de pontos internos da grade, em cada direção
n = 50

In [ ]:
# Comprimento das células da grade (dx, dy)
h1 = (I[1] - I[0])/(n + 1)
h2 = (J[1] - J[0])/(n + 1)

## Discretização da equação de Laplace

In [ ]:
# Quantidade de incógnitas
m = n**2
m

## Grade de diferenças finitas

In [ ]:
x = np.linspace(I[0], I[1], n+2)
y = np.linspace(J[0], J[1], n+2)
X, Y = np.meshgrid(x, y)
plt.scatter(X,Y)
plt.show()

## Identificação do contorno

In [ ]:
def contorno(i,j,n): # n é a quantidade de nós internos da grade
    if (i == 0) or (i == (n+1)) or (j == 0) or (j == (n+1)):
        return True
    return False

## Numeração das equações

In [ ]:
def equacao(i,j,n): # n é a quantidade de nós internos da grade
    return (i-1)*n + (j-1)

## Aplicando as condições de contorno

In [ ]:
# Condição de contorno
solucao = np.zeros((n+2,n+2))

def f(y):
    if 0 <= y <= 1:
        return y
    else:
        return 2.0 - y

y = np.linspace(J[0], J[1], n+2)
for i in range(n+2):
    solucao[i, n+1] = f(y[i])

## Montagem do sistema linear esparso

In [ ]:
# Contagem de coeficientes não nulos
nnz = 0
for i in range(1,n+1):
    for j in range(1,n+1):
        # Contabiliza o coeficiente da diagonal
        nnz = nnz + 1

        # Índices dos nós vizinhos
        vizinhos = [(i+1, j), (i-1, j), (i, j+1), (i, j-1)]
        for k,v in enumerate(vizinhos):
            if not contorno(v[0], v[1], n):
                nnz = nnz + 1        

print("nnz =", nnz)

## Montagem do sistema linear esparso

In [ ]:
# Montagem do sistema no formato CSR
csr_linha  = np.zeros((m+1), dtype=int)
csr_coluna  = np.zeros((nnz,), dtype=int)
csr_valor  = np.zeros((nnz,))
bcsr = np.zeros((m,))

## Montagem do sistema linear esparso

In [ ]:
cont = 0 # usado para preencher os arranjos das colunas e valores
prox = 0 # usado para preencher o arranjo das linhas
pesos = [-1.0/h2**2, -1.0/h1**2, 2.0*(1.0/h1**2 + 1.0/h2**2), -1.0/h1**2, -1.0/h2**2]
for i in range(1,n+1):
    for j in range(1,n+1):
        # Número da equação/incógnita
        linha = equacao(i, j, n)

        # Nós do stencil em ordem crescente dos números das equações correspondentes
        nos = [(i-1, j), (i, j-1), (i,j), (i, j+1), (i+1, j)]
        for k,v in enumerate(nos):
            if not contorno(v[0], v[1], n):
                coluna = equacao(v[0], v[1], n)
                csr_coluna[cont] = coluna
                csr_valor[cont] = pesos[k]
                cont = cont + 1
            else:
                bcsr[linha] = bcsr[linha] - pesos[k]*solucao[v[0], v[1]]

        prox = prox + 1
        csr_linha[prox] = cont

## Padrão de esparsidade

In [ ]:
Acsr = csr_matrix((csr_valor, csr_coluna, csr_linha))
plt.spy(Acsr, markersize = 0.5)
plt.show()

## Resolvendo o sistema linear esparso

In [ ]:
%%time
# Resolvendo o sistema
u = sp.gmres(Acsr,bcsr)[0]
solucao[1:n+1, 1:n+1] = u.reshape(n,n)

## Solução via diferenças finitas

In [ ]:
# Plotando a solução da equação de Laplace
fig, ax = plt.subplots()
mapa = ax.pcolormesh(X, Y, solucao, cmap=cm.coolwarm)
fig.colorbar(mapa, ax=ax)
plt.show()

## Solução via diferenças finitas

In [ ]:
# Plotando a solução da equação de Laplace
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(X, Y, solucao, cmap=cm.coolwarm,linewidth=0, antialiased=False)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

## Referências

* [Notas de aula](https://www-users.cse.umn.edu/~saad/csci8314/) do professor Yousef Saad da disciplina "CSCI 8314: Sparse matrix computations".

* [Notas de aula](https://courses.grainger.illinois.edu/cs357/sp2024/) da professora Mariana Silva da disciplina "CS 357 - Numerical Methods I".

## Referências

* SAAD, Yousef. Iterative methods for sparse linear systems. Society for Industrial and Applied Mathematics, 2003.

* BERTACCINI, Daniele; DURASTANTE, Fabio. Iterative methods and preconditioning for large and sparse linear systems with applications. Chapman and Hall/CRC, 2018.

## Referências

* O exemplo da equação de Laplace foi extraído dos slides do professor Marcos Eduardo Valle sobre a [técnica de separação de variáveis](https://www.ime.unicamp.br/~valle/Teaching/2016/MA311/Aula27.pdf).

* As matrizes contidas neste repositório no formato [MatrixMarket](https://math.nist.gov/MatrixMarket/) foram obtidas no site [SuiteSparse](https://suitesparse-collection-website.herokuapp.com/).

**Introdução às matrizes esparsas**

Vicente Sobrinho

I Encontro Regional de Matemática  
UFCA | Centro de Ciências e Tecnologia  
<font size=4pt>Juazeiro do Norte, 7 de outubro de 2024</font>